## Text Network

### Install dependencies

In [3]:
import re
import itertools
import spacy
from nltk import FreqDist, bigrams
import numpy as np
import pandas as pd
import networkx as nx
from pyvis.network import Network
import seaborn as sns
from collections import Counter
sns.set(rc={'figure.figsize':(15,7)})


from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from youtube_transcript_api import YouTubeTranscriptApi
from gensim.models import KeyedVectors

In [40]:
def generate_transcript(id):
	transcript = YouTubeTranscriptApi.get_transcript(id,languages=['en'])
	script = ""

	for text in transcript:
		t = text["text"]
		if t != '[Music]':
			script += t + " "
		
	return script, len(script.split())

iden = 'rTbKpSTYxPI'

#ID é a parte final do link, depois do "=", exemplo 
#https://www.youtube.com/watch?v=L_jWHffIx5E
#L_jWHffIx5E

transcript, no_of_words = generate_transcript(iden)
print(transcript,no_of_words)

Just short of two years after King Charles 
I raised his banner at Nottingham in the   summer of 1642, his cause was dealt a potentially 
mortal sucker punch due to the Scottish Covenanter   intervention and the subsequent disaster at 
Marston Moor. Thousands of badly needed royalist   soldiers lay dead and the entirety of Northern 
England was now under parliamentary control.   However, although it seemed like everything was 
tipping in the roundheads’ favour, parliament   did have its problems. However, many short-term 
triumphs they enjoyed, it was clear that temporary   local fighters and militiamen could never achieve 
a truly permanent victory against the king.   Welcome to our fourth video on the English 
Civil War, parliament’s establishment of the   New Model Army and its first proper test against 
Charles’ final dice throw at Naseby in 1645.  It really used to be hard to stake your claim 
to lands in the British Isles, but if you want   to do it now and reap some rewards for 

In [41]:
transcript2 = YouTubeTranscriptApi.get_transcript(iden,languages=['en'])

# dicionário do momento de cada fala
legend_dict = {}
for text in transcript2:
    legend_dict[text['start']] = text['text']
    


In [25]:

modelo = KeyedVectors.load_word2vec_format("C:/Users/rafae/Downloads/MindTube-main (2)/MindTube-main/notebook/wiki-news-300d-1M.vec")

KeyboardInterrupt: 

### Auxiliary functions

In [42]:
toktok = ToktokTokenizer()
stpwrd = stopwords.words('english')
stop_list = (['uh', 'also', 'oh', 'um', 'yeah', 'use', 'lot', 'put', 'get', 'would', 'gonna', 'really', 'much', 'actually', 'another'])
stpwrd.extend(stop_list)

'''for word in stpwrd:
    print(word)
    '''
    
    
#Acha as N palavras mais similares a string passada e retorna uma lista com as palavras. (Sem a métrica de proximidade)
def getSimilar(string, N):
    similarList = modelo.most_similar(positive=[string])[0:N]
    wordList = []
    for word in similarList:
        wordList.append(word)
    return wordList

def sort_list(list1, list2):
     
    list3 = []
    for entry in list2:
        list3.append(entry[1])
    zipped_pairs = zip(list3, list1)
 
    z = [x for _, x in sorted(zipped_pairs, reverse = True)]
 
    return z



#Busca na label dos nós por palavras similares a stopwords
def banStopword(lista):
    SUSwords = []
    similarWords = []
    for word in lista:
        similares = getSimilar(word,5)
        mostSimilar = 0
        similarWord = ''
        SUSWord = ''
        for palavra in similares:
            sentinel = 0
            if palavra[0] in stpwrd and palavra[1] > mostSimilar:
                mostSimilar = palavra[1]
                similarWord = palavra
                SUSWord = word
                sentinel = 1
                print(mostSimilar, similarWord, SUSWord)
            if sentinel == 1:  
                SUSwords.append(word)
                similarWords.append(palavra)

    return SUSwords, similarWords

'''word_tokens = toktok.tokenize(example_sent)

filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

filtered_sentence = []

for w in word_tokens:
	if w not in stop_words:
		filtered_sentence.append(w)

print(word_tokens)
print(filtered_sentence)'''


def filtered_sentence(example_sent):
    word_tokens = toktok.tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stpwrd]
    return " ".join(filtered_sentence)

def remove_stopwords(text):
    doc = filtered_sentence(text)
    print(" ".join([token.text for token in doc if token.is_stop == False]))
    return " ".join([token.text for token in doc if token.is_stop == False])

def pre_processing(text):
    text = filtered_sentence(text)
    text = re.sub(r'[^\w\s]', '', text) #remove punctuation
    text = re.sub(r"\s{2,}", " ", text)
    return text

def co_occurrence(text):
    corpus = list(itertools.chain.from_iterable([text.split(' ')]))
    vocab = list(set(corpus))
    vocab_index = {word: i for i, word in enumerate(vocab)}
    bi_grams = list(bigrams(corpus))
    bigram_freq = FreqDist(bi_grams).most_common(len(bi_grams)) 
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab))) 
    
    #bigram = ((word n, word n+1), num_occurrences)
    for bigram in bigram_freq:
        current = bigram[0][1] #word n+1
        previous = bigram[0][0] #word n
        count = bigram[1] #num_occurrences
        pos_current = vocab_index[current] #obtain id for word n+1 
        pos_previous = vocab_index[previous] #obtain id for word n 
        co_occurrence_matrix[pos_current][pos_previous] = count
        
    return co_occurrence_matrix, vocab_index

def structure_text_network(matrix, vocab_index):
    data_matrix = pd.DataFrame(matrix, index=vocab_index, columns=vocab_index)
    data_stack = data_matrix.stack()
    structure = data_stack[data_stack >= 1].rename_axis(('source', 'target')).reset_index(name='weight')
    return structure[(structure.source != " ") & (structure.target != " ")]

In [43]:
lista = ['one', 'measurements', 'what', 'also', 'me', 'mice', 'five', 'standard', 'stat', 'imagine', 'spread', 'weighed', 'look', 'mean', 'another', 'quest', 'quantifies', 'often', 'let', 'plot', 'set', 'deviation', 'data', 'measured', 'differences']

lista1, lista2 = banStopword(lista)

0.7335886359214783 ('another', 0.7335886359214783) one
0.6885168552398682 ('how', 0.6885168552398682) what
0.7364087700843811 ('a', 0.7364087700843811) another


In [44]:
print(lista1)
print('\n',lista2)
sort_list(lista1, lista2)

['one', 'what', 'another']

 [('another', 0.7335886359214783), ('how', 0.6885168552398682), ('a', 0.7364087700843811)]


['another', 'one', 'what']

In [45]:
getSimilar('one',5)

[('another', 0.7335886359214783),
 ('only', 0.7045549750328064),
 ('each', 0.6934645771980286),
 ('a', 0.6761848330497742),
 ('either', 0.6739281415939331)]

### Generate text network

In [46]:
from spacy.lang.pt.stop_words import STOP_WORDS
nlp = spacy.load("pt_core_news_sm")
#STOP_WORDS.add('gotta')

In [47]:
text = transcript

In [48]:
text_cleaned = pre_processing(text)
matrix, vocab = co_occurrence(text_cleaned)
structure_network = structure_text_network(matrix, vocab)

In [49]:
structure_network

,source,target,weight
0,blocked,October,1.0
1,Dust,deploying,1.0
2,Dust,horse,1.0
3,soninlaw,Cromwell,1.0
4,Speenhamland,9th,1.0
...,...,...,...
2199,diminished,far,1.0
2200,diminished,king,1.0
2201,amazing,title,1.0
2202,consider,Please,1.0


In [50]:
text_network = nx.DiGraph()
text_network = nx.from_pandas_edgelist(structure_network, 'target', 'source', ['weight'])

### PageRank

In [51]:
text_network = nx.DiGraph()
text_network = nx.from_pandas_edgelist(structure_network, 'target', 'source', ['weight'])

In [52]:
def pagerank_centrality(G):
    centrality =  nx.pagerank(G)
    return sorted(centrality.items(), key=lambda x: x[1], reverse=False)

In [53]:
centrality = pagerank_centrality(text_network)

#dicionário da centralidade
cen_dict = {}
max_cen = 0
for x in centrality:
    cen_dict[x[0]] = x[1]
    if max_cen < x[1]:
        max_cen = x[1]

### Graph Visualization

In [54]:
# lista dos nós
name_lst = [x[0] for x in centrality]

# lista de nós a serem removidos, estamos mantendo os 25 maiores
rmv_lst = name_lst[:-25]

# exibição do nome dos nós para a visualização
for i in text_network.nodes:
    text_network.nodes[i]['label'] = i

# redução do grafo mantendo a conectividade
for i in rmv_lst:
    maxi = 0
    for j in text_network.neighbors(i):
        w = name_lst.index(j)
        if maxi < w:
            maxi = w
            merge_node = j
    text_network.remove_node(i)

In [60]:
# momentos em que cada palavra é dita
dict_moments = {}
episilon = 45

for i in text_network.nodes:
    dict_moments[i] = []
    
for i in list(legend_dict.keys()):
    phrase = pre_processing(legend_dict[i])
    
    for j in text_network.nodes:
        if j in phrase.split() and (len(dict_moments[j]) == 0 or  dict_moments[j][-1]+episilon<i):
            dict_moments[j].append((i>1)*(int(i)-1))
            print(f"{j} was appended at {i}")
    
dict_moments['trade']        

Charles was appended at 5.28
Covenanter was appended at 11.88
royalist was appended at 18.78
soldiers was appended at 24.96
parliament was appended at 30.96
However was appended at 30.96
king was appended at 42.3
Army was appended at 53.16
Model was appended at 53.16
Charles was appended at 53.16
New was appended at 53.16
foot was appended at 93.36
Rupert was appended at 147.9
Charles was appended at 152.34
parliamentarian was appended at 158.28
parliament was appended at 169.44
royalist was appended at 176.04
king was appended at 181.98
army was appended at 212.88
Charles was appended at 212.88
cavalry was appended at 219.96
Army was appended at 232.14
parliament was appended at 232.14
king was appended at 237.54
parliamentarian was appended at 237.54
royalists was appended at 266.58
Charles was appended at 271.56
soldiers was appended at 276.42
war was appended at 284.52
Cromwell was appended at 296.58
army was appended at 317.1
king was appended at 321.48
parliament was appended at 

KeyError: 'trade'

In [61]:
nt = Network('800px', '1000px', notebook=True)
nt.from_nx(text_network)
for edg in nt.edges:
    # espessura e física das arestas
    #edg['width'] = text_network[i][j]['weight']**4
    #if edg['weight'] < 1:
    
    if edg['from'] == edg['to']:
        edg['hidden'] = True
    edg['physics'] = False

for n in nt.nodes:
    # tamanho dos nós
    n['shape'] = 'ellipse'
    size = 40*(cen_dict[n['label']]/max_cen)**0.4
    n['font'] = str(int(size))+'px arial white'
    n['color'] = '#7f333f'
    n['labelHighlightBold'] = True
    

def time_to_min(time):
    return str(time//60)+":"+"0"*(time%60<10)+str(time%60)

for n in nt.nodes:
    # links nos nós
    link_list = ["<a href='https://www.youtube.com/watch?v="+str(iden)+
                 "&t="+str(time)+"s' target='_blank' rel='noopener noreferrer'>"+time_to_min(time)+"<br>" for time in dict_moments[n['label']]]
    soma = ''
    for k in link_list:
        soma += k
    n['title'] = soma
    #print(n['size'])

    
nt.show("text_network.html")

In [59]:
#print(nx.adjacency_matrix(text_network))
#print(nt.nodes)
lista = []
for node in nt.nodes:
    lista.append(node['label'])
print(lista)

['horse', 'foot', 'parliamentarian', 'Cromwell', 'parliament', 'Fairfax', 'war', 'began', 'However', 'royalist', 'army', 'soldiers', 'cavalry', 'Charles', 'Covenanter', 'king', 'Army', 'Model', 'New', 'infantry', 'made', 'Rupert', 'Goring', 'royalists', 'Montrose']


In [ ]:
SUS = banStopword(lista)
print(SUS)